<a href="https://colab.research.google.com/github/margeumkim/BRIDGEMAIL/blob/master/04_word_sense_disambiguation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT 

##1. Connecting to a GPU and mounting a google drive folder


In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install ipython-autotime
%load_ext autotime
import pandas as pd 
import re

# Import the parsed email data 
email_directory = "/content/drive/My Drive/data/emails_parsed.csv"
emails_df  = pd.read_csv(email_directory)
keyword_dict_directory = "/content/drive/My Drive/data/keywords_reduced.csv"  #### long list = final_keywords_dictionary.csv
keyword_dict  = pd.read_csv(keyword_dict_directory)   
keyword_dict = keyword_dict.sort_values(by='mean', ascending=False)
keyword_dict_100 = keyword_dict[0:100]  # Keeping the top 100 context-dependent words
address_user_directory = "/content/drive/My Drive/data/address_user_df.csv"
address_user_df  = pd.read_csv(address_user_directory)


## 2. Importing BERT pretrained model and tokenizer and defining functions



In [ ]:
!pip install transformers

import torch
from transformers import *
import pdb
import operator
from collections import OrderedDict
import sys

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

PATH='bert-base-uncased'
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(PATH,do_lower_case=True)
model = BertForMaskedLM.from_pretrained(PATH)
model.eval()

def get_sent():
    #print("Enter sentence:")
    sent = my_sentence
    if (not sent.endswith(".")):
#        print("Appending period to do dummy masking")
        sent = sent + " ."
    return '[CLS] ' + sent + '[SEP]'

def print_tokens(tokenized_text):
    dstr = ""
    for i in range(len(tokenized_text)):
        dstr += "   " +  str(i) + ":"+tokenized_text[i]
    print(dstr)
    print()


def get_pos():
    while True:
        masked_index = 0
        try:
                masked_index = int(my_index)
                return masked_index
        except:
            print("Enter valid number: (0 to quit)")
            masked_index = int(my_index)
            if (masked_index == 0):
                print("Quitting")
                sys.exit()
            return masked_index


## 3. Subsetting into a user, find sentences that contains a keyword

In [ ]:
dict_columns = ['user', 'keyword', 'high_att_words']
user_word_meaning_dict_df = pd.DataFrame(columns=dict_columns)

# First layer: Over user
for l in range(len(address_user_df)):      
  my_user = address_user_df.iloc[l]["user"]     
  my_emails = emails_df[emails_df["user"] == my_user]
  #len(my_emails)

  my_sentences = []
  
  user_word_meaning_dict = []  # this will store my outputs

  for index, row in my_emails.iterrows():
      if type(row['content']) ==  str:
        for j in range(len(row['content'].split('.'))):
          my_sentences.append(row['content'].split('.')[j])
      else:
        #print (index)
        pass
  
  my_sentences = pd.DataFrame(my_sentences)
  

  # Second layer: For each user, iterate over keywords 
  for m in range(len(keyword_dict_100)):                               

    keyword = [keyword_dict_100.iloc[m]['Term'].split('\'')[1]]            

    my_sentences.columns = ['Sentence']

    sentence_for_keyword = []

      # Third layer: For keyword, let's find sentences that contain the keyword
    for index, row in my_sentences.iterrows():

        if type(row['Sentence']) == str:
      
            if any(item in list(row['Sentence'].split(' ')) for item in keyword):
            #print (my_bool)
              sentence_for_keyword.append(row['Sentence'])
    
    #print(len(sentence_for_keyword))
    if len(sentence_for_keyword) > 10:
     
      # I will store top attention words for the keyword in this list
      keyword_meaning = []

      # Fourth layer: Now that we know the sentences, let's evaluate each sentence using pretrained BERT. And extract top attention words
      for k in range(len(sentence_for_keyword)):

        my_sentence = sentence_for_keyword[k]

        my_breaker = True
        text = get_sent()

        tokenized_text = tokenizer.tokenize(text)    # BERT takes "tokens" (index associated with each word) instead of words. This step tokenizes my sentence

        # Fifth layer: Some sentences have just too many words/tokens to use BERT. Skip those sentences
        if len(tokenized_text) > 500:    
          pass

        else:
          
          # Some keywords does not exist in the BERT's dictionary. 
          # For example,  = ['fax']  does not exist in BERT's dictionary -- Tokenizer breakes it down to : fa / ##x; 
          # I am skipping these cases
          try: 
            my_index = tokenized_text.index(keyword[0]) 
          except: 
            pass
          #print_tokens(tokenized_text)

          indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)


          # Create the segments tensors.
          segments_ids = [0] * len(tokenized_text)
          masked_index = len(tokenized_text) - 2
          tokenized_text[masked_index] = "[MASK]"
          indexed_tokens[masked_index] = 103
          results_dict = {}

          # Convert inputs to PyTorch tensors
          tokens_tensor = torch.tensor([indexed_tokens])
          segments_tensors = torch.tensor([segments_ids])

          with torch.no_grad():
              predictions = model(tokens_tensor, segments_tensors)

          # Sixth layer: While within top 10 range, extract high attention words              
          while (my_breaker == True):
              #print_tokens(tokenized_text)
              #print("Enter any term position neighbor:")
              masked_index = get_pos()
              results_dict = {}

              # Some sentences are from the headings from a forwarded messages. I will ignore these headings
              if (masked_index) <= len(tokenized_text):
                for i in range(len(predictions[0][0,masked_index])):
                    tok = tokenizer.convert_ids_to_tokens([i])[0]
                    results_dict[tok] = float(predictions[0][0,masked_index][i].tolist())

                k = 0
                hist_d = {}
                sorted_d = OrderedDict(sorted(results_dict.items(), key=lambda kv: kv[1], reverse=True))
                first = True
                max_val = 0
                for i in sorted_d:
                    if (first):
                        max_val = sorted_d[i]
                        first = False
                    val = round(float(sorted_d[i])/max_val,1)
                    if (val in hist_d):
                        hist_d[val] += 1
                    else:
                        hist_d[val] = 1
                    k += 1
        #            if (k <= 10):    # If you activate this if statement, you will be able to see top 10 attention words 
        #                pass
                        #print(i,sorted_d[i])    
                    if (k > 10):
                      my_breaker = False

              # Append the top 10 words in terms of the attention scores to the keyword_meaning list
              for j in range(10):
                keyword_meaning.append(list(list(sorted_d.items())[j]))
              
    keyword_meaning_df = pd.DataFrame(keyword_meaning)
    keyword_meaning_df.columns = ['keyword', 'attention']
    keyword_meaning_df = keyword_meaning_df[keyword_meaning_df['keyword'] != keyword[0]]    # Often, the keyword itself is the top attention word. Let's remove those instances
    keyword_meaning_df = keyword_meaning_df.groupby('keyword',as_index=False).mean()    # If a attention word was extracted from many different sentences (instances), take the mean of attention scores from each instances.
    keyword_meaning_df = keyword_meaning_df.sort_values(by='attention', ascending=False).reset_index()
    #keyword_meaning_df.head(20)

    # BERT sometimes splits a word into pieces and use ## to remark such cases. Since tokens like ##er or pre## do not give much context to users, I am removing them from the keyword_meaning list. 
    drop_list = []
    for i in range(len(keyword_meaning_df)):
      if len(re.findall(r'##', keyword_meaning_df['keyword'][i])) > 0:
        drop_list.append(1)
      else: 
        drop_list.append(0)    

    keyword_meaning_df['drop_flag'] = drop_list
    keyword_meaning_df = keyword_meaning_df[keyword_meaning_df['drop_flag'] == 0]

    # Store the final top 10 attention words to a list
    my_keyword_meaning_top10 = keyword_meaning_df[0:10][['keyword']]
    user_word_meaning_dict.append([my_user, keyword[0], list(my_keyword_meaning_top10['keyword'])])
    #print(m)

  # Store the list to pickle so that we can access the information using streamlit 
  #print(my_user)
  user_word_meaning_dict_df = user_word_meaning_dict_df.append(pd.DataFrame(user_word_meaning_dict, columns = dict_columns))

filename_to_pickle = "/content/drive/My Drive/data/word_meaning.pkl"
user_word_meaning_dict_df.to_pickle(filename_to_pickle) 

